# 一等函数

Everything in Python is an object.

本章的内容较为松散，主要介绍了Python中函数的一些重要特性。

Python中的整数、字符串、字典甚至函数都是一等对象（First-class Object）。它们有共同的特点：

1. 在运行时创建和销毁
2. 能赋值给变量或者作为数据结构中的元素
3. 能够作为参数传递给函数
4. 能够作为函数的返回结果

Python中的函数同样是一个对象，其类型是"function"。在Python中，可以把函数赋值给变量，然后通过变量名进行调用；也可以作为参数传递给其他函数，这些行为和一般的整数、字符串没有本质上的差别。

In [3]:
def test_func():
    print("Everything in Python is an object.")

type(test_func)

function

## 高阶函数（higher-order function）

高阶函数：接收函数作为参数或者将函数作为返回结果的函数。

Python内置一些高阶函数，类似于map、filter、sorted以及reduce等，这些函数均接受另一个函数作为参数传入。

此外，将函数作为参数传入可以实现固定流程中具体处理方法的自定义，例如总流程是：步骤1->步骤2->步骤3，但是每一步的具体操作在不同应用场合中不同，此时就可以传入特定的函数以实现不同效果。

# 作为函数的对象以及作为对象的函数

## 对象与可调用对象

Python中一些对象可以表现得像一个普通函数。可调用对象可以使用调用运算符(())来实现调用。可调用对象有如下几类：

1. 自定义的函数
2. 内置函数
3. 内置方法
4. 方法
5. 类（创建实例时）
6. 类的实例（要求实现\_\_call__方法）
7. 生成器函数（yield）


Python中任何对象可以表现得像一个函数（可调用）。对于Python，只要实现了\_\_call__方法，就可以使用调用运算符对实例进行调用

## 函数内省（function introspection）

同样的，Python的函数也可以被视为对象。在Python中，函数有很多属性，这一特点使得Python中的函数表现得像一个对象。通过函数内省(dir())能够得到这些属性的列表。

### 函数参数信息的获取与分析

函数内省可以用来获取以及分析函数参数。例如在一个完整的请求处理机制中，可以通过函数内省快速分析某一函数需求哪些参数，然后从请求中获取这些参数并传递给该函数。

Python的函数有大量可用于获取函数参数信息的内置方法:

* .__defaults__: 获取参数的默认值
* .__code__.co_varnames: 参数名称，包括函数中创建的临时变量
* .__code__.co_argcount: 参数总数

上述方法能够获取函数参数信息，但是不易处理和后续分析。inspect模块则提供了快速分析函数参数的功能。inspect.signature()可以直接返回有序映射形式的参数名以及其默认值，此外这个方法还能有效解析参数的类型（仅限关键字参数、关键字参数字典等）。除了获取参数信息外，inspect还可以使用bind方法将任意个参数绑定到函数的形参上。

In [6]:
class Add:

    def add(self, x, y):
        return x + y
    
    def __call__(self, x, y):
        return self.add(x, y)

my_add = Add()
print("使用实例的方法计算1+2：", my_add.add(1, 2))
print("直接调用实例计算1+2：", my_add(1, 2))

使用实例的方法计算1+2： 3
直接调用实例计算1+2： 3


## 函数的特性

### 仅限关键字参数（keyword-only argument）

Python按照如下流程传参(参考[PEP 3102](https://peps.python.org/pep-3102/))：

1. 为每一个形参分配一个参数槽（slot）
2. 初始化所有参数槽，并标记为空"empty"
3. 定位参数/位置参数（positional argument）首先被赋值，然后是关键字参数
4. 对于每一个positional argument
    * 首先尝试按照顺序将传入的值绑定到空的参数槽。若该参数槽不是一个可变形参（varargs argument）的参数槽（*描述的形参）则将该参数槽标记为"filled"
    * 若该参数槽是一个可变形参，并且没有命名（单仅有一个 *），Raise Error
    * 否则将剩余的positional argument全部放入可变形参对应的参数槽
5. 对于每一个关键字参数
    * 若该关键字是函数的一个形参，则直接将传入的值赋予该关键字参数槽，并且将该参数槽标注为“filled”。若该关键字槽已经被标注为“filled”，则Raise Error
    * 否则，若存在"keyword dictionary"形参（**描述的形参），将传入的关键字记入该形参。若"keyword dictionary"形参中已经记录了该关键字，则直接Raise Error
    * 若与上述两种情况均不匹配，直接Raise Error
6. 最后
    * 对于可变形参的参数槽，若没有数据填充该槽则直接将空列表作为该形参的值
    * 对于没有赋值的槽，若该槽有默认参数则使用默认参数填充该槽，否则Raise Error

仅限关键字参数实际上就发生在上述流程中的第4步。按照第4步的正常流程，所有的形参均会被按顺序赋值。但是有时候不希望这种默认的赋值方式，而是希望有些参数一定要用关键字参数进行赋值并且不允许依照positional argument的方式对该形参进行赋值。

仅限关键字参数就是基于这种需求提出的，这一形参位于可变形参之后，这使得positional argument永远无法对该关键字进行赋值。

以下为对[PEP 3102](https://peps.python.org/pep-3102/)描述的传参流程进行测试

In [3]:
def test_func_0(arg_1, key_only_arg=None):

    print("arg_1: {}".format(arg_1))
    print("key_only_arg: {}".format(key_only_arg))

def test_func_1(arg_1, key_only_arg=None, *args):

    print("arg_1: {}".format(arg_1))
    print("key_only_arg: {}".format(key_only_arg))

    if args:
        print("args: ", end="")
        for i in args:
            print("{}\\".format(i), end=" ")
    else:
        print("")
    print("")

def test_func_2(arg_1, *args, key_only_arg=None):

    print("arg_1: {}".format(arg_1))
    print("key_only_arg: {}".format(key_only_arg))

    if args:
        print("args: ", end="")
        for i in args:
            print("{}\\".format(i), end=" ")
    else:
        print("")
    print("")

def test_func_3(arg_1, *args, key_only_arg=None, **kwargs):

    print("arg_1: {}".format(arg_1))

    if key_only_arg is not None:
        kwargs["key_only_arg"] = key_only_arg
    
    if kwargs:
        for key in kwargs:
            print("{}:{}".format(key, kwargs[key]))

    if args:
        print("args: ", end="")
        for i in args:
            print("{}\\".format(i), end=" ")
    else:
        print("")
    print("")


# ----------------- 正常运行test_func_0 ----------------- 
# 完全按照positional argument的方式进行传参
print("\n正常运行，步骤4第一种情况")
arg_1 = "test_func"
key_only_arg = "check"
test_func_0(arg_1, key_only_arg)

# ----------------- 错误运行test_func_0 ----------------- 
# 完全按照positional argument的方式进行传参
# 但是传入的实参多于形参
print("\n错误运行，步骤4第二种情况")
arg_1 = "test_func"
key_only_arg = "check"
try:
    test_func_0(arg_1, key_only_arg, "more args")
except TypeError:
    print("实参多于形参")

# ----------------- 正确运行test_func_1 ----------------- 
# 完全按照positional argument的方式进行传参
# 但是传入的实参多于形参
print("\n正确运行，步骤4第三种情况")
arg_1 = "test_func"
key_only_arg = "check"
try:
    test_func_1(arg_1, key_only_arg, "more args")
except TypeError:
    print("实参多于形参")

# ----------------- 正确运行test_func_2 ----------------- 
# 完全按照positional argument的方式进行传参
# 但是由于key_only_arg是仅限关键字参数，key_only_arg没有被赋值
print("\n正确运行，但是key_only_arg没有被赋值")
arg_1 = "test_func"
key_only_arg = "check"
try:
    test_func_2(arg_1, key_only_arg, "more args")
except TypeError:
    print("实参多于形参")

# ----------------- 正确运行test_func_2 ----------------- 
# 按照positional argument的方式进行传参，
# key_only_arg按照关键字传参
print("\n正确运行，key_only_arg被赋值")
arg_1 = "test_func"
key_only_arg = "check"
try:
    test_func_2(arg_1, "more args", key_only_arg=key_only_arg)
except TypeError:
    print("实参多于形参")


# ----------------- 错误运行test_func_2 ----------------- 
# 按照positional argument的方式进行传参，
# key_only_arg按照关键字传参，但是重复赋值
print("\n错误运行，多余的关键字参数，步骤5第三种情况")
arg_1 = "test_func"
key_only_arg = "check"
try:
    test_func_2(arg_1, "more args", 
    key_only_arg=key_only_arg, kwags_1="check_2")
except TypeError:
    print("多余的关键字参数")



正常运行，步骤4第一种情况
arg_1: test_func
key_only_arg: check

错误运行，步骤4第二种情况
实参多于形参

正确运行，步骤4第三种情况
arg_1: test_func
key_only_arg: check
args: more args\ 

正确运行，但是key_only_arg没有被赋值
arg_1: test_func
key_only_arg: None
args: check\ more args\ 

正确运行，key_only_arg被赋值
arg_1: test_func
key_only_arg: check
args: more args\ 

错误运行，多余的关键字参数，步骤5第三种情况
多余的关键字参数


In [34]:
# ----------------- 错误运行test_func_2 ----------------- 
# 按照positional argument的方式进行传参，
# key_only_arg按照关键字传参，但是重复赋值
print("\n错误运行，重复关键字")
arg_1 = "test_func"
key_only_arg = "check"
try:
    test_func_2(arg_1, "more args", 
    key_only_arg=key_only_arg, key_only_arg="check_2")
except SyntaxError:
    print("同一关键字重复传参")

SyntaxError: keyword argument repeated (Temp/ipykernel_33880/2613957010.py, line 9)

不同于其他的传参问题raise TypeError，重复传参会raise SyntaxError，并且无法被try捕捉。这一现象和[PEP 3102](https://peps.python.org/pep-3102/)中的描述不太相符（In accordance with the current Python implementation, any errors encountered will be signaled by raising **TypeError**. (If you want something different, that’s a subject for a different PEP.)）目前还没找到对应这一现象的PEP

### 函数注解

Python允许对函数的形参以及函数本身进行注释。例如标注某一参数的类型为"int"或者"str"。但是值得注意的是，Python不会对这些注释进行任何处理，**不会根据注释对参数进行检查、强制处理或者验证**，仅是存储到\_\_annotations__属性中，以供IDE（例如为IDE的静态类型检查功能提供信息）、框架和装饰器使用。

## Python中的函数式编程

Guido van Rossum不认为Python是一种函数式编程语言，但是Python也对函数式编程有一些支持。这部分有需要的时候再阅读。

## 总结

* Everything in Python is an object.
* Python中的函数是一等公民。Python允许将函数赋值给变量、作为参数传递给其他函数、作为元素存储在数据结构中以及任意创建和销毁函数对象并访问函数的属性。
* 高阶函数为一些实现提供了便利。作为将其他函数作为参数的函数，高阶函数在项目中能实现更灵活的功能。
* Python中有多种可调用对象，并且可以通过实现\_\_call__方法使得任何自定义类成为可调用对象。
* Python函数有相对灵活的参数处理机制，在positional argument、可变参数、关键字参数、关键字参数字典以及仅限关键字参数的配合下能够实现相当灵活的传参功能。
* 函数内省使得获取和分析函数参数信息非常方便，在inspect模块的支持下，可以很方便的获取参数信息。
* 函数注解提供了关于函数本身以及函数参数更为丰富的信息，能够为IDE静态类型检查等功能提供额外信息。值得注意的是，这些函数注解不会对函数的行为造成任何影响，Python解释器不会自动根据函数注解对参数进行检查或者验证。